In [29]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

## About Data

Twitter has become an important communication channel in times of emergency.
The ubiquitousness of smartphones enables people to announce an emergency they’re observing in real-time. Because of this, more agencies are interested in programatically monitoring Twitter (i.e. disaster relief organizations and news agencies).

But, it’s not always clear whether a person’s words are actually announcing a disaster. Take this example:
![image.png](assets/tweet_screenshot.png)

The author explicitly uses the word “ABLAZE” but means it metaphorically. This is clear to a human right away, especially with the visual aid. But it’s less clear to a machine.

-------
Columns: 

id - a unique identifier for each tweet

text - the text of the tweet

location - the location the tweet was sent from (may be blank)

keyword - a particular keyword from the tweet (may be blank)

target - in train.csv only, this denotes whether a tweet is about a real disaster (1) or not (0)


## Data Prepration

### Reading data

In [30]:
df_train = pd.read_csv(r'data\train.csv')
test = pd.read_csv(r'data\test.csv')

### Investigating the dataset

In [31]:
def background_color(value):
    if isinstance(value, str):
        return 'background-color: #a6c0ed'
    return ''

def show_df(df_train):
    print('shape'.center(30,'_'))
    display(df_train.shape)

    print('head'.center(30,'_'))
    display(df_train.head().style.background_gradient(cmap='Blues'))

    print('tail'.center(30,'_'))
    display(df_train.tail().style.background_gradient(cmap='Blues'))

    print('info'.center(30,'_')+'\n')
    display(df_train.info())

    print('describe_continuous'.center(30,'_'))
    display(df_train.describe().T.style.background_gradient(cmap = 'Blues'))

    print('describe_categorical'.center(30,'_'))
    display(df_train.describe(include='object').T.style.background_gradient(cmap='Blues'))

    print('null_values_percent'.center(30,'_'))
    display((df_train.isna().sum() / len(df_train) * 100).sort_values(ascending=False))
show_df(df_train)

____________shape_____________


(7613, 5)

_____________head_____________


,id,keyword,location,text,target
0,1,nan,nan,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,nan,nan,Forest fire near La Ronge Sask. Canada,1
2,5,nan,nan,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,nan,nan,"13,000 people receive #wildfires evacuation orders in California",1
4,7,nan,nan,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


_____________tail_____________


...

,count,mean,std,min,25%,50%,75%,max
id,7613.000000,5441.934848,3137.116090,1.000000,2734.000000,5408.000000,8146.000000,10873.000000
target,7613.000000,0.429660,0.495060,0.000000,0.000000,0.000000,1.000000,1.000000


_____describe_categorical_____


,count,unique,top,freq
keyword,7552,221,fatalities,45
location,5080,3341,USA,104
text,7613,7503,11-Year-Old Boy Charged With Manslaughter of Toddler: Report: An 11-year-old boy has been charged with manslaughter over the fatal sh...,10


_____null_values_percent______


location    33.272035
keyword      0.801261
id           0.000000
text         0.000000
target       0.000000
dtype: float64

### Preprocessing

#### Removing URL's from the dataset

In [32]:
import re
import string

def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)


In [33]:
print("Text Before:\n", df_train.text[32])
print("Text After:\n",remove_URL(df_train.text[32]))

Text Before:
 We always try to bring the heavy. #metal #RT http://t.co/YAo1e0xngw
Text After:
 We always try to bring the heavy. #metal #RT 


In [34]:
# df_train = pd.DataFrame({'text' : ['Im going to school', 'How are you', "Im Ahmed"]})

#### Removing punctuations from the dataset

In [35]:
def remove_punct(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [36]:
print("Text Before:\n", df_train.text[2])
print("Text After:\n",remove_punct(df_train.text[2]))

Text Before:
 All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected
Text After:
 All residents asked to shelter in place are being notified by officers No other evacuation or shelter in place orders are expected


#### Removing stop words

In [37]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [38]:
# Stop words example
stopwords.words("english")[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [39]:
def remove_stopwords(text):
    stop = set(stopwords.words("english"))
    
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)

In [40]:
print("Text Before:\n", df_train.text[2])
print("Text After:\n",remove_stopwords(df_train.text[2]))

Text Before:
 All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected
Text After:
 residents asked 'shelter place' notified officers. evacuation shelter place orders expected


#### Now let's apply our cleaning methods

In [41]:
df_train["text"] = df_train.text.map(remove_URL) # map(lambda x: remove_URL(x))
df_train["text"] = df_train.text.map(remove_punct)
df_train["text"] = df_train.text.map(remove_stopwords)
df_train["text"]

0            deeds reason earthquake may allah forgive us
1                   forest fire near la ronge sask canada
2       residents asked shelter place notified officer...
3       13000 people receive wildfires evacuation orde...
4       got sent photo ruby alaska smoke wildfires pou...
                              ...                        
7608    two giant cranes holding bridge collapse nearb...
7609    ariaahrary thetawniest control wild fires cali...
7610                      m194 0104 utc5km volcano hawaii
7611    police investigating ebike collided car little...
7612    latest homes razed northern california wildfir...
Name: text, Length: 7613, dtype: object

-------------

### Tokenization

In [42]:
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [43]:
df_train["text"] = df_train.text.map(nltk.tokenize.word_tokenize)
df_train.text.head()

0    [deeds, reason, earthquake, may, allah, forgiv...
1        [forest, fire, near, la, ronge, sask, canada]
2    [residents, asked, shelter, place, notified, o...
3    [13000, people, receive, wildfires, evacuation...
4    [got, sent, photo, ruby, alaska, smoke, wildfi...
Name: text, dtype: object

### Lemmatizing the tokens

In [44]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [45]:
def Lemmatize(sentence_tokens):
    lemmatizer = WordNetLemmatizer()
    result_sentence = []
    for token in sentence_tokens:
        result_sentence.append(lemmatizer.lemmatize(token))
    return result_sentence

In [46]:
df_train["text"] = df_train.text.map(Lemmatize)
df_train.text.head()

0    [deed, reason, earthquake, may, allah, forgive...
1        [forest, fire, near, la, ronge, sask, canada]
2    [resident, asked, shelter, place, notified, of...
3    [13000, people, receive, wildfire, evacuation,...
4    [got, sent, photo, ruby, alaska, smoke, wildfi...
Name: text, dtype: object

## Calculate Probabilty with N-grams

In [47]:
from nltk.util import ngrams

### Adding start and end tokens

In [48]:
df_train['text'] = df_train.text.map(lambda x: ["<S>"] + x + ["<E>"])
df_train.text.head()

0    [<S>, deed, reason, earthquake, may, allah, fo...
1    [<S>, forest, fire, near, la, ronge, sask, can...
2    [<S>, resident, asked, shelter, place, notifie...
3    [<S>, 13000, people, receive, wildfire, evacua...
4    [<S>, got, sent, photo, ruby, alaska, smoke, w...
Name: text, dtype: object

#### Calculate tokens frequencies

In [49]:
# Flatten the token as putting them all in single list
def flatten_tokens(df):
    tokens = [token for item in df for token in item]      
    return tokens

In [50]:
tokens = flatten_tokens(df_train.text)
tokens_fd = nltk.FreqDist(tokens)
tokens_fd.most_common(10)

[('<S>', 7613),
 ('<E>', 7613),
 ('fire', 350),
 ('like', 347),
 ('im', 299),
 ('amp', 298),
 ('get', 255),
 ('u', 246),
 ('new', 224),
 ('via', 220)]

#### Get Bigrams and calculate their frequencies

In [51]:
bigrams = df_train.text.apply(lambda x:list(ngrams(x, 2)))
bigrams[:5]

0    [(<S>, deed), (deed, reason), (reason, earthqu...
1    [(<S>, forest), (forest, fire), (fire, near), ...
2    [(<S>, resident), (resident, asked), (asked, s...
3    [(<S>, 13000), (13000, people), (people, recei...
4    [(<S>, got), (got, sent), (sent, photo), (phot...
Name: text, dtype: object

In [52]:
# Flatten the bigrams as putting them all in single list
def flatten_bigrams(df):
    bigrams = [bigram for item in df for bigram in item]
    return bigrams

In [53]:
flattened = flatten_bigrams(bigrams)
len(flatten_bigrams(bigrams))

79384

In [54]:
fd_bi = nltk.FreqDist(flattened)
fd_bi.most_common(10)

[(('<S>', 'new'), 74),
 (('<S>', 'im'), 70),
 (('suicide', 'bomber'), 60),
 (('burning', 'building'), 58),
 (('fire', '<E>'), 58),
 (('news', '<E>'), 53),
 (('\x89û', '<E>'), 50),
 (('look', 'like'), 49),
 (('body', 'bag'), 48),
 (('<S>', 'rt'), 47)]

### Estimating the probabiltiy of a word sequence
P(x1, x2, ..., xn) = P(x1)P(x2|x1)...P(xn|x1,...xn-1)

In [55]:
def prob_calc(sentence):
    probs = []
    for bigram in sentence:
        # Probabilty of current bigram = frequency(bigram) / (frequency(first token of bigram) == number of words prefix to second token of bigram)
        prob = fd_bi[bigram] / tokens_fd[bigram[0]] 
        probs.append(prob)
        print(f"P({bigram[0]} | {bigram[1]}) = {prob:.3}")
    print("="*40)
    
    res = probs[0] 
    sen = " ".join([word[0] for word in sentence])
    print(f"P({sen}) = ({probs[0]:.4})",end="")
    for prob in probs[1:]:
        res *= prob
        print(f" * ({prob:.4})", end= "")
    print(f" = {res:.2}")
    

### Print the likely hood of the first 10 senteces

In [56]:
for bigram in bigrams[0:10]:
    prob_calc(bigram)
    print("\n")

P(<S> | deed) = 0.000131
P(deed | reason) = 0.5
P(reason | earthquake) = 0.0323
P(earthquake | may) = 0.0189
P(may | allah) = 0.0341
P(allah | forgive) = 0.111
P(forgive | u) = 0.5
P(u | <E>) = 0.0813
P(<S> deed reason earthquake may allah forgive u) = (0.0001314) * (0.5) * (0.03226) * (0.01887) * (0.03409) * (0.1111) * (0.5) * (0.0813) = 6.2e-12


P(<S> | forest) = 0.000657
P(forest | fire) = 0.424
P(fire | near) = 0.0114
P(near | la) = 0.0185
P(la | ronge) = 0.0357
P(ronge | sask) = 1.0
P(sask | canada) = 1.0
P(canada | <E>) = 0.231
P(<S> forest fire near la ronge sask canada) = (0.0006568) * (0.4242) * (0.01143) * (0.01852) * (0.03571) * (1.0) * (1.0) * (0.2308) = 4.9e-10


P(<S> | resident) = 0.000394
P(resident | asked) = 0.125
P(asked | shelter) = 0.111
P(shelter | place) = 0.333
P(place | notified) = 0.0323
P(notified | officer) = 1.0
P(officer | evacuation) = 0.027
P(evacuation | shelter) = 0.0192
P(shelter | place) = 0.333
P(place | order) = 0.0323
P(order | expected) = 0.0286